# Modules and Classes

In [ ]:
import dataclasses
import datetime as dt
from pathlib import Path
import sys

# Make script files importable
SCRIPTS_DIR = Path.cwd().parent / 'scripts'
sys.path.insert(0, str(SCRIPTS_DIR.absolute()))

## Modules

Modules are Python files that can be imported.  Importing a module executes the Python in the file.

In [ ]:
import plot_climate_data_good as pcdg

In [ ]:
# We can access variables and functions
print(f"DATA_DIR defined in module: {pcdg.DATA_DIR}")
station_file_path = Path('some_stationdata.txt')
print(f"Station name from file path: {pcdg.get_station_name(station_file_path)}")

The methods can also be combined with data to make a class-based version of the code.
We'll look at how this works later.

In [ ]:
from class_version.historic_station_data import MetOfficeHistoricStationData

station_data = MetOfficeHistoricStationData(pcdg.DATA_DIR / 'sheffielddata.txt')
print(f"Station name: {station_data.station_name}")
print(f"Mean max temp: {station_data.mean_maximum_temperature()}")
print(f"Data statistics:\n{station_data.station_data.describe()}")


In [ ]:
station_data.plot_max_temp_figure()

## Classes for data

Classes are another way to group variables (attributes) and functions (methods).
Everything in Python is a class.
`dir` shows the attributes and methods that are present.
The private "__dunder__" methods determine class behaviour within Python.

In [ ]:
my_integer = 1
print(f"type: {type(my_integer)}")
dir(my_integer)

A simple class needs a function (`__init__`) to initialise it.
A `__repr__` function defines how it is printed.
The methods of the class instance take `self` as the first parameter which allows them to access internal data.

Classes represent things, so their names are **nouns**, in _camelCase_.

In [ ]:
class Person:
    def __init__(self, name: str):
        self.name = name

    def __repr__(self):
        return f"Person({self.name})"


person = Person('John')
print(f"person: {person}")
print(f"type: {type(person)}")
dir(person)

The attributes of a class are accessible via `.` notation.

In [ ]:
print(person.name)


### Exercise

+ Give the person an `age` attribute
+ Create a person, then change their `name` and `age`

### Dataclasses

Dataclasses are a convenient way to store data with a __repr__ and __init__ and with type hints to help keep your code right.
This example is similar to a row from a database table.

In [ ]:
@dataclasses.dataclass
class MonitoringStation:
    sensor_type: str
    sensor_serial_number: int
    longitude: float
    latitude: float
    deploy_date: dt.date

station = MonitoringStation(
    sensor_type="Guralp 2000",
    sensor_serial_number=1234,
    longitude=-3.1,
    latitude=54.0,
    deploy_date=dt.date(2023, 1, 20)
)

print(station)

# Attributes are accessed and updated using . notation
print(f"serial number: {station.sensor_serial_number}")
station.sensor_serial_number = 5678
print(f"new serial number: {station.sensor_serial_number}")

# Dataclasses can conveniently be turned into dictionaries or tuples
print(f"asdict: {dataclasses.asdict(station)}")

Classes can have methods that act on their attributes.

In [ ]:
@dataclasses.dataclass
class MonitoringStation:
    sensor_type: str
    sensor_serial_number: int
    longitude: float
    latitude: float
    deploy_date: dt.date

    def uptime(self) -> int:
        """Return time since deploy date in days"""
        delta = dt.date.today() - self.deploy_date
        return delta.days

station = MonitoringStation(
    sensor_type="Guralp 2000",
    sensor_serial_number=1234,
    longitude=-3.1,
    latitude=54.0,
    deploy_date=dt.date(2023, 1, 20)
)

print(f"uptime: {station.uptime()} days")

Classes can keep a record of past states with custom updates.

In [ ]:
@dataclasses.dataclass
class MonitoringStation:
    sensor_type: str
    sensor_serial_number: int
    longitude: float
    latitude: float
    deploy_date: dt.date

    def __post_init__(self):
        # Register
        self.previous_deployments: list[tuple[float, float, dt.date]] = []
        self.deploy(self.longitude, self.latitude, self.deploy_date)

    def uptime(self) -> int:
        """Return time since deploy date in days"""
        delta = dt.date.today() - self.deploy_date
        return delta.days

    def deploy(
        self, longitude: float, latitude: float, deploy_date: dt.date = dt.date.today()
    ):
        """Record a new deployment of the sensor."""
        # Store old deployment
        current_deployment = (self.longitude, self.latitude, self.deploy_date)
        self.previous_deployments.append(current_deployment)

        # Update current attributes
        self.longitude = longitude
        self.latitude = latitude
        self.deploy_date = deploy_date
    
    # A property is a method that takes only "self" as an argument and
    # behaves like an (read only in this case) attribute.
    @property
    def deployments_count(self):
        return len(self.previous_deployments)


station = MonitoringStation(
    sensor_type="Guralp 2000",
    sensor_serial_number=1234,
    latitude=54.0,
    longitude=-3.1,
    deploy_date=dt.date(2023, 1, 20)
)

station.deploy(52, -2, dt.date(2024, 1, 12))
station.deploy(63, -14, dt.date(2024, 6, 2))
print(f"deployments count: {station.deployments_count}")
print(f"previous deployments: {station.previous_deployments}")

Inheritance allows classes to reuse code.

In [ ]:
@dataclasses.dataclass
class SeismicMonitoringStation(MonitoringStation):
    high_pass_filter_frequency: float


@dataclasses.dataclass
class GeoMagMonitoringStation(MonitoringStation):
    magnet_type: str


seismic_station = SeismicMonitoringStation(
    sensor_type="Guralp 2000",
    sensor_serial_number=1234,
    latitude=54.0,
    longitude=-3.1,
    deploy_date=dt.date(1993, 1, 20),
    high_pass_filter_frequency=45.3
)

geomag_station = GeoMagMonitoringStation(
    sensor_type="MagTastic SuperPro",
    sensor_serial_number=9999,
    latitude=54.0,
    longitude=-3.1,
    deploy_date=dt.date(2003, 3, 5),
    magnet_type="Inductive Pickup"
)

print(f"seismic uptime: {seismic_station.uptime()} days")
print(f"geomag deployments: {geomag_station.deployments_count}")

### Exercise

+ Look at the `plot_climate_data_classy.py` file.  How does it compare to the `plot_climate_data_good.py`?
+ Why does `historic_station_data.py` not have a `__name__ == "__main__"` section?

### Collections of classes

Custom classes can be used to store your own classes.
Internally, they are backed by a container class e.g. a list or dictionary.


In [ ]:
class MonitoringStationCatalog:
    """
    A catalog for storing monitoring station metadata.
    """
    def __init__(self):
        self._stations: dict = {}

    def register(self, station: MonitoringStation):
        self._stations[station.sensor_serial_number] = station

    def unregister(self, serial_number: int):
        self._stations.pop(serial_number)

    @property
    def station_count(self):
        return len(self._stations)

In [ ]:
catalog = MonitoringStationCatalog()
catalog.register(seismic_station)
catalog.register(geomag_station)

print(catalog.station_count)

In [ ]:
catalog.unregister(1234)
print(catalog.station_count)

### Exercise

+ Add a `locations()` function that returns a dictionary of data, with serial_number as the key and a tuple of (longitude, latitude) as the value.
+ Note what happens to the locations() result when you redeploy any of the stations.  This is a good example of `mutable` data.

#### LLM-assisted exercise

Enter the following prompt into your favourite LLM and use the result to add a `write_locations_to_geojson` method to your class.

>
> Write a Python function that takes a dictionary of `serial_number` and (`lon`, `lat`) coordinates and
> converts it into a GeoJSON file.  The signature of the function should be:
>
> def write_locations_to_geojson(locations: dict, filename: str = 'stations.geojson'):
>
> The data in the locations dictionary is of the form: `{'ABC1': (-3.1, 54)}`
>

Using LLMs at the level of a function is good because you can check and understand the output.